# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção: (caso o site esteja preparado para evitar esse tipo de utilizaçao do selenium)

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [1]:
import pandas as pd

produtos_df = pd.read_excel('arquivos/buscas.xlsx')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch 11,3000,3500
1,rtx 3060,zota galax,4000,4500


In [3]:
from selenium import webdriver # Comandar navegador controlado
from selenium.webdriver.chrome.service import Service # responsável por iniciar e parar o chromedriver
from webdriver_manager.chrome import ChromeDriverManager # Gerenciar arquvio ChromeDriver

# Instalar arquivo temporário ChromeDrive a partir da versão atual do Chrome
servico = Service(ChromeDriverManager().install())

# Abrir navegador com tela maximizada
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Criar navegador
nav = webdriver.Chrome(service=servico, options=options)

In [36]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Criando Function Para Listar Valores Encontrados Google Shopping

def pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar google shopping
    nav.get('https://shopping.google.com/?nord=1')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)

    # Filtrar Valor min
    nav.find_element(By.NAME, 'lower').send_keys(str(preço_min))
    # Filtrar Valor max
    nav.find_element(By.NAME, 'upper').send_keys(str(preço_max))
    # Aplicar filtro de valor
    nav.find_element(By.NAME, 'upper').send_keys(Keys.ENTER)
        
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'a8Pemb').text

            # Selecionando elemento com link do produto
            elemento_link_filho = elemento.find_element(By.CLASS_NAME, 'aULzUe')
            elemento_link_pai = elemento_link_filho.find_element(By.XPATH, '..')
            link = elemento_link_pai.get_attribute('href')
            
            resultado_pesquisa.append((nome_produto, preço, link))

    # Ordenando lsta crescentemente a partir do indice 1 da tupla
    resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
    return resultado_pesquisa


In [92]:
from selenium.webdriver import ActionChains # Performar ação double click
import time

# Criando Function Para Listar Valores Encontrados Buscape

def pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar google shopping
    nav.get('https://www.buscape.com.br/')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
  
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard__1D3ve')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            # Filtrando Valores
            if preço_min <= preco <= preço_max:

                # Selecionando elemento com link do produto
                link = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb').get_attribute('href')
            
                resultado_pesquisa.append((nome_produto, preço, link))

    # Ordenando lsta crescentemente a partir do indice 1 da tupla
    resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
    return resultado_pesquisa

In [91]:
info_produto = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard__1D3ve')

for elemento in info_produto:
    nome_produto = elemento.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text.lower()
    print(nome_produto)


smartphone apple iphone 12 64gb câmera dupla
smartphone apple iphone 12 mini 64gb câmera dupla
smartphone apple iphone 12 usado 64gb câmera dupla
smartphone apple iphone 12 vermelho 64gb câmera dupla
smartphone apple iphone 12 mini vermelho 64gb câmera dupla


In [93]:
valores_encontrados = pd.DataFrame()

for linha in produtos_df.index:
    nomes_banidos = produtos_df.loc[linha, 'Termos banidos']
    nome_produto = produtos_df.loc[linha, 'Nome']
    preço_max = produtos_df.loc[linha, 'Preço máximo']
    preço_min = produtos_df.loc[linha, 'Preço mínimo']

    # gshop = pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    # if gshop:
    #     gshop_df = pd.DataFrame(gshop, columns=['PRODUTO', 'PREÇO', 'LINK'])
    #     valores_encontrados = pd.concat([valores_encontrados, gshop_df])
    # else:
    #     gshop_df = None

    buscape = pesquisa_buscape(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    if buscape:
        buscape_df = pd.DataFrame(buscape, columns=['PRODUTO', 'PREÇO', 'LINK'])
        valores_encontrados = pd.concat([valores_encontrados, buscape_df])
    else:
        buscape_df = None

display(valores_encontrados)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B0A813+48355]
	(No symbol) [0x00A9C4B1]
	(No symbol) [0x009A5358]
	(No symbol) [0x009A87A1]
	(No symbol) [0x009A99E1]
	(No symbol) [0x009A9A80]
	(No symbol) [0x009D061C]
	(No symbol) [0x009D0B3B]
	(No symbol) [0x009C9AE1]
	(No symbol) [0x009EA784]
	(No symbol) [0x009C9A36]
	(No symbol) [0x009EAA94]
	(No symbol) [0x009FC922]
	(No symbol) [0x009EA536]
	(No symbol) [0x009C82DC]
	(No symbol) [0x009C93DD]
	GetHandleVerifier [0x00D6AABD+2539405]
	GetHandleVerifier [0x00DAA78F+2800735]
	GetHandleVerifier [0x00DA456C+2775612]
	GetHandleVerifier [0x00B951E0+616112]
	(No symbol) [0x00AA5F8C]
	(No symbol) [0x00AA2328]
	(No symbol) [0x00AA240B]
	(No symbol) [0x00A94FF7]
	BaseThreadInitThunk [0x76867D59+25]
	RtlInitializeExceptionChain [0x76F1B74B+107]
	RtlClearBits [0x76F1B6CF+191]


In [5]:
print(produtos_df['Preço mínimo'][0])

3000
